# Data Preprocessing - Advanced Statistics Topics

In this notebook, we do the general preprocessing and exploratory data analysis of our dataset.

## Import Libreries

In this section we import the requiere libraries and establish the connection with Google Drive.

In [1]:
# Se importan las librerias necesarias
import numpy as np
import pandas as  pd
import os
import gzip
import timeit
from google.colab import drive

# Tensorflow 2.x
import tensorflow as tf # Not required for this notebook 
print('TENSORFLOW VERSION: {}'.format(tf.__version__))

TENSORFLOW VERSION: 2.4.1


In [2]:
# Se fija el directorio referencia con el se conecta colab a drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Se guarda en una variable el directorio del curso

# Directorio de los datos
directorio_data = 'drive/MyDrive/Colab Notebooks/AST-ImageCaptioning/Data/'

## Procesamiento de Datos

En esta sección se procesan los datos con los cuales se va a realizar el experimento de: __CPU__ vs __GPU__ vs __TPU__.

In [ ]:
# Funcion para cargar datos mnist (carga imagen y tag)
def load_mnist(ruta, tipo='train'):
    ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
    ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)
    
    with gzip.open(ruta_categorias, 'rb') as rut_cat:
        etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

    with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
        imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

    return imagenes, etiquetas

In [ ]:
# Carga los datos de entrenamiento y prueba
X_train, Y_train = load_mnist(directorio_data, tipo='train')
X_test, Y_test = load_mnist(directorio_data, tipo='test')

# Se hace un recorte para que la cantidad de datos sea multiplo de 128 (cuestion de lo que acepta usar la TPU)
multiplo_train = (X_train.shape[0] // 128)*128
multiplo_test = (X_test.shape[0] // 128)*128
X_train, Y_train = X_train[:multiplo_train,:], Y_train[:multiplo_train]
X_test, Y_test = X_test[:multiplo_test,:], Y_test[:multiplo_test]

# Se hace un reshape para la data correspondiente a los pixeles de la imagen
dims = (28, 28, 1)
X_train, X_test = np.reshape(X_train, (X_train.shape[0], dims[0], dims[1], dims[2])), np.reshape(X_test, (X_test.shape[0], dims[0], dims[1], dims[2]))

## Modelo de Clasificación: Red Neuronal Convolucional

En esta sección se define la arquitectura de la CNN que se a usar para clasificar las imágenes entre las 10 categorías de productos disponibles.

In [ ]:
# No es necesario correr esta celda cuando se v a a realizar el experimento para TPU

# Modelo de CNN a usar

# Semilla aleatoria fijada 
tf.random.set_seed(200)

# Capa General 1: Capa Convolucional
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

# Capa General 2: Capa Convolucional
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

# Capa General 3: Capa Convolucional
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

# Capa General 4: Capa Final (Aplanamiento, Densa y Softmax)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))

# Descomente si desea ver la arquitectura de la CNN
# model.summary()

# Se especifica cual es el optimizador (metodo de descenso del gradiente a usar, la función de perdida y las metricas a mostrar)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

## Comparativa en CPU, GPU y TPU

En esta sección se realiza la comparativa entre los tiempos que demora en promedio una época de entrenamiento para cada tipo PU (unidad de procesamiento). Se corrió el notebook tres veces y en cada una se especifico en __Colab__ cual PU usar, la celda que obtiene el tiempo promedio por epoca de una PU se corrió únicamente cuando esa PU estaba especificada.

### CPU 

En esta subsección se realiza la prueba para la __CPU__.

In [ ]:
# Se especifican cuantas epocas de prueba
epocas = 2

# Una funcion para entrenar el modelo con los parametros especificados
def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=128, epochs=epocas, verbose=1)
  return None

# Tiempo el cual se demora en total
cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')

# Printea el tiempo promedio por epoca
print('EL TIEMPO PROMEDIO POR EPOCA DE ENTRENAMIENTO PARA LA CPU ES DE: {} SEGUNDOS'.format(np.round(cpu_time/epocas, 3)))

Epoch 1/2
468/468 [==============================] - 771s 2s/step - loss: 1.0169 - accuracy: 0.7078 - val_loss: 0.4264 - val_accuracy: 0.8551
Epoch 2/2
468/468 [==============================] - 770s 2s/step - loss: 0.4105 - accuracy: 0.8562 - val_loss: 0.3127 - val_accuracy: 0.8922
EL TIEMPO PROMEDIO POR EPOCA DE ENTRENAMIENTO PARA LA CPU ES DE: 770.537 SEGUNDOS


### GPU

En esta subsección se realiza la prueba para __GPU__.

In [ ]:
# Se verifica que se este usando la GPU
nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU NO ENCONTRADA')
print('GPU ENCONTRADA: {}'.format(nombre_gpu))

GPU ENCONTRADA: /device:GPU:0


In [ ]:
# Se especifican cuantas epocas de prueba
epocas = 2

# Una funcion para entrenar el modelo con los parametros especificados
def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=128, epochs=epocas, verbose=1)
  return None

# Tiempo el cual se demora en total
gpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu')

# Printea el tiempo promedio por epoca
print('EL TIEMPO PROMEDIO POR EPOCA DE ENTRENAMIENTO PARA LA GPU ES DE: {} SEGUNDOS'.format(np.round(gpu_time/epocas, 3)))

Epoch 1/2
468/468 [==============================] - 21s 41ms/step - loss: 1.0141 - accuracy: 0.7087 - val_loss: 0.3789 - val_accuracy: 0.8668
Epoch 2/2
468/468 [==============================] - 19s 40ms/step - loss: 0.4069 - accuracy: 0.8566 - val_loss: 0.3427 - val_accuracy: 0.8838
EL TIEMPO PROMEDIO POR EPOCA DE ENTRENAMIENTO PARA LA GPU ES DE: 19.844 SEGUNDOS


### TPU 

En esta subsección se realiza la prueba para __TPU__.

In [ ]:
# Con esto se configura la TPU y se revisa que efectivamente este siendo usada
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])  # Detectar TPU
  print('TPU encontrada ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: TPU no encontrada!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# Se usa la TPU para definir la arquitectura de la CNN
with tpu_strategy.scope():
  # Modelo de CNN a usar

  # Semilla aleatoria fijada 
  tf.random.set_seed(200)

  # Capa General 1: Capa Convolucional
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  # Capa General 2: Capa Convolucional
  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  # Capa General 3: Capa Convolucional
  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  # Capa General 4: Capa Final (Aplanamiento, Densa y Softmax)
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))

  # Se especifica cual es el optimizador (metodo de descenso del gradiente a usar, la función de perdida y las metricas a mostrar)
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Se cambia el tipo de los datos a np.float 32
x_train = X_train.astype(np.float32)
y_train = Y_train.astype(np.float32)
x_test = X_test.astype(np.float32)
y_test = Y_test.astype(np.float32)

# Se especifican cuantas epocas de prueba
epocas = 2

# Una funcion para entrenar el modelo con los parametros especificados
def entrenamiento_tpu():
  model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=epocas, verbose=1)
  return None

# Tiempo el cual se demora en total
tpu_time = timeit.timeit('entrenamiento_tpu()', number=1, setup='from __main__ import entrenamiento_tpu')

# Printea el tiempo promedio por epoca
print('EL TIEMPO PROMEDIO POR EPOCA DE ENTRENAMIENTO PARA LA TPU ES DE: {} SEGUNDOS'.format(np.round(tpu_time/epocas, 3)))

Epoch 1/2
468/468 [==============================] - 14s 20ms/step - loss: 1.0472 - accuracy: 0.7021 - val_loss: 0.3664 - val_accuracy: 0.8695
Epoch 2/2
468/468 [==============================] - 8s 17ms/step - loss: 0.4114 - accuracy: 0.8545 - val_loss: 0.3200 - val_accuracy: 0.8872
EL TIEMPO PROMEDIO POR EPOCA DE ENTRENAMIENTO PARA LA TPU ES DE: 12.079 SEGUNDOS


### Resultados

Los resultados que se encontraron son: 

1. __CPU__: 770.527 segundos (12.84 minutos aproximadamente) por epoca en promedio.

2. __GPU__: 19.844 segundos por epoca en promedio.

3. __TPU__: 12.079 segundos por epoca en promedio.  

El resultado es el esperado y la __TPU__ toma el menor tiempo en promedio para entrenar la CNN (usando 2 epocas como referencia, se podría tratar con más epocas para __GPU__ y __TPU__).